# 🟩 커스텀 데이터셋 제작 공부하기       

https://tutorials.pytorch.kr/beginner/basics/data_tutorial.html  


<br>

## 🟢 **[Dataset 이해하기]** Dataset 클래스를 상속받아서 내가 직접 __len__과 __getitem__을 정의  

In [46]:
import torch #훈련
from torch.utils.data import Dataset, DataLoader #커스텀 데이터셋을 만들기 위함

import os #로컬 컴퓨터에서 데이터셋을 불러오기 위해
import cv2 # 시각화 해서 표현하기 위함

In [47]:
# Dataset 클래스를 상속받아서 새로운 Dataset을 정의

# __함수__ : 던더 함수
class SimpleDataset(Dataset):
    # 이 클래스의 객체가 생성되면 무조건 __init__ 함수가 동작하게 됨
    # 기초 데이터, 함수 동작하도록 연결
    def __init__(self, t):
        self.t = t

    # 데이터셋의 길이 반환
    # 파이토치가 훈련을 할 때 len() -> 배치사이즈 대비 얼마나 데이터셋이 남아있는 지 체크할 때 씀
    #    len(dataset) 을 실행하면 이 함수가 호출됨
    def __len__(self):
        return self.t

    # 데이터 + 라벨 -> 한 쌍을 뽑는 데 씀
    # idx(index) = 몇 번째 데이터?
    #    dataset[0] 처럼 특정 데이터를 꺼낼 때 실행됨
    def __getitem__(self, idx):
        # 여기서는 단순히 인덱스 번호를 LongTensor(정수형 텐서)로 반환
        return torch.LongTensor([idx])

Dataset 클래스  
   │  
   ├── __init__(데이터 준비)  
   │  
   ├── __len__() → 데이터 개수 알려줌  
   │  
   └── __getitem__(i) → i번째 데이터 꺼내줌  


In [48]:
# SimpleDataset 클래스를 이용해 총 7개의 데이터가 있는 데이터셋을 만듭니다.
dataset = SimpleDataset(t=7)

dataset.__len__()

7

In [49]:
len(dataset)  # 7 (데이터 개수)

7

In [50]:
dataset.__getitem__(3)

tensor([3])

In [51]:
dataset[3]  # tensor([3])  (세 번째 데이터 꺼냄)

tensor([3])

In [52]:
# 🆘🔥🔥🔥🔥🔥 무한대로 나오는 이유를 모르겠습니다.
# for i, data in enumerate(dataset):
#     print(f"{i}번째 : {data}")

In [53]:
type(dataset[3])

torch.Tensor

<br>

## 🟢 **[DataLoader 이해하기]** - 데이터셋을 훈련에 사용하기 좋게 만들기  

만약 데이터가 100만 개라면, 이걸 한 번에 처리하기는 어렵다. 그래서 DataLoader가 batch_size만큼 데이터를 잘라서 조금씩 모델에 전달해주는 역할을 합니다.  

In [54]:
# 이 데이터셋을 훈련에서 어떻게 활용할 것인가?
dataloader = DataLoader(
    dataset=dataset,    # 어떤 데이터셋을 훈련에 쓸 거니?
    batch_size=2,       # 데이터셋을 쪼개 배치로 만듦
    shuffle=True,       # 데이터셋을 섞어주는(랜덤)
    drop_last=True,     # 배치사이즈 만들고 남은 데이터 쓸거야? (True: 버림)
    # 총 7개의 데이터를 batch_size 2개씩 짝지어줬을 때, 나머지 1개가 남은 것을 버립니다.
)

In [55]:
dataloader

In [56]:
# DataLoader가 데이터를 어떻게 배치(batch) 단위로 나누어 주는지 학습과정을 이해하기 쉬운 예시

# epoch: 전체 데이터를 모두 사용해서 한 번 학습하는 과정

for e in range(3):  # 총 3번의 epoch동안 훈련을 한다.
    print(f"epoch : {e}")   # 현재 몇번째 epoch 인지 출력.

    for batch in dataloader:    # dataset 전체를 돌면서 학습합니다.
        print(batch)
        print(type(batch))
        print("\n")

epoch : 0
tensor([[4],
        [2]])
<class 'torch.Tensor'>


tensor([[0],
        [6]])
<class 'torch.Tensor'>


tensor([[5],
        [3]])
<class 'torch.Tensor'>


epoch : 1
tensor([[0],
        [4]])
<class 'torch.Tensor'>


tensor([[3],
        [6]])
<class 'torch.Tensor'>


tensor([[2],
        [5]])
<class 'torch.Tensor'>


epoch : 2
tensor([[1],
        [4]])
<class 'torch.Tensor'>


tensor([[3],
        [0]])
<class 'torch.Tensor'>


tensor([[6],
        [5]])
<class 'torch.Tensor'>




<br>

---
---
---

## 🟢 **[CustomData 만들기]** - (예시) 이미지 데이터셋을 커스텀화 함          
- 이제 본격적으로 이미지 파일을 불러와서 처리하는 나만의 데이터셋 클래스 만들기!!!  

- 이미지(0~255) 숫자로 이루어진 데이터 -> 딥러닝 -> 텐서 형태로 표현  
- 텐서 형태로 이미지를 표현해 줘야 함.  

In [57]:
# torchvision: PyTorch에서 이미지 처리/데이터셋 관련 기능을 제공하는 라이브러리
# 🔥transforms(전처리): 이미지의 크기를 조절하거나, 딥러닝 모델이 이해할 수 있는 텐서(Tensor) 형태로 변환하는 등 (자르기, 회전, 텐서 변환, 정규화 등)
#                       이미지 전처리(preprocessing) 기능을 담당합니다.
from torchvision import transforms 

# PIL(Pillow) 라이브러리의 Image: 파이썬에서 이미지를 불러오고 다룰 때 사용하는 기본적인 도구입니다.
from PIL import Image

import matplotlib.pyplot as plt # 이미지 시각화

In [ ]:
class CustomData(Dataset):
    def __init__(self, path, transform):
        self.image_path = path  # 전체 이미지 경로를 저장
        # self.image_path는 보통 문자열 경로들의 리스트(예: ["data/a.jpg","data/b.jpg", ...])
        # 또는 pathlib.Path 객체들의 리스트로 기대됩니다. "path"라는 이름 때문에 폴더 경로 하나를 받을 것처럼 보일 수 있으니
        # 실제로는 '파일 경로 리스트'인지 '폴더 경로(내부 탐색 필요)'인지 명확히 해야 합니다.
        
        self.transform = transform  # 이미지에 대해 일괄적으로 적용할 '전처리'
        # transform은 callable (예: torchvision.transforms.Compose([...]))이어야 하며,
        # 입력으로 PIL.Image을 받고 출력으로 torch.Tensor(또는 변환된 PIL/ndarray)를 반환하는 것이 일반적입니다.

    def __len__(self):
        # if (이미지수 == 라벨수) return 이미지 수
        return len(self.image_path)
        # __len__은 DataLoader가 전체 데이터 개수를 알기 위해 호출합니다.
        # image_path가 리스트가 아니라면(예: glob 결과가 빈 경우) 여기서 에러 또는 0을 반환할 수 있습니다.


    # 텐서 형태로 변환된 이미지를 돌려줌
    def __getitem__(self, idx):
        image = self.image_path[idx]
        # image 변수는 '파일 경로 문자열'입니다. 주석과 혼동하면 안 됩니다(이미지가 아님).
        # 예: image == "data/img_001.jpg"

        image = Image.open(image)  # 해당 경로에서 이미지를 읽어 옴
        # PIL.Image.open은 파일 핸들을 연 상태로 반환(지연 로딩)합니다.
        # 파일을 안전하게 닫으려면 with Image.open(path) as img: 같은 문법을 권장합니다.
        # 또한, 채널 통일을 위해 .convert('RGB') 혹은 .convert('L')을 호출하는 것이 안전합니다.

        if self.transform is not None:
            result = self.transform(image)
            # transform이 ToTensor 등을 포함하면 result는 torch.Tensor (C,H,W), dtype=float32, 값범위 0~1 이 됩니다.
            # 주의: transform이 PIL 이미지를 변경하고 닫지 않으므로 이미지 리소스 관리를 신경 써야 합니다.

        # csv,파일 이름 등을 이용해서 데이터에 맞는 라벨을 반환
        label = 1
        # 현재는 하드코딩된 더미 라벨(1)입니다. 실제 작업에서는:
        # - 파일명/폴더 구조에서 라벨을 파싱하거나,
        # - __init__에 labels 리스트/딕셔너리를 전달하거나,
        # - CSV 파일을 읽어 라벨 맵을 만들어 사용해야 합니다.
        # 라벨은 정수(int) 또는 torch.Tensor (예: torch.tensor(label, dtype=torch.long)) 로 반환하는 것이 일반적입니다.

        return result, label
        # 주의: 만약 self.transform이 None이면 result 변수가 정의되지 않아 NameError가 납니다.
        # 따라서 transform None 케이스를 반드시 처리해야 합니다.

<br><br>

---

## 🟢 `CustomData`에 넣을 `인수` or `매개변수` 정의하기  


### 🟡 **[transforms.Compose]** - 이미지 전처리 방법 정의하기  
- 만들어놓은 CustomData class를 사용하기 전에 전처리 방법을 정의하자.  
- 딥러닝 모델에 이미지를 넣기 전에, 모든 이미지의 크기를 같게 만들고, 딥러닝 모델이 계산하기 좋은 형태로 바꿔주는 등의 전처리 과정이 필요.  

In [59]:
# 여러 가지 전처리 방법을 하나로 묶어주는 transforms.Compose를 사용합니다.

transform = transforms.Compose(
    [
        # 1. 이미지 크기 조절: 모든 이미지의 크기를 224x224 픽셀로 맞춥니다.
        transforms.Resize((224, 224)),

        # 2. 텐서(Tensor)로 변환: 이미지를 딥러닝 모델이 계산할 수 있는 숫자 행렬(텐서)로 바꿉니다.
        transforms.ToTensor(),

        # 3. 정규화(Normalize): 이미지의 픽셀 값 범위를 조정하여 모델이 더 빠르고 안정적으로 학습하도록 돕습니다.
        #    아래 mean과 std 값은 ImageNet 데이터셋에서 미리 계산된 값으로, 보통 그대로 많이 사용합니다.
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)



### 🟡 **[path 지정]** - (예시) 이미지 파일 경로 리스트 만들기  (❌ 데이터 없음)  
- 🔥 path들을 리스트로 만들어한다는 것  

In [60]:
origin = "./data/safety-data/human-accident" # 이미지 파일이 모여있는 directory를 가져옵니다.

os.listdir(origin)  # 🔥 origin으로 정의한 directory에 존재하는 것들(ex. 이미지들, 영상들, 등등)을 리스트로 만들어줍니다!!!

['bump', 'fall-down', 'fall-off', 'hit', 'jam', 'no-accident']

In [61]:
# os.listdir(origin)로 만든 리스트의 요소 하나씩을 가져와 origin path 뒤에 붙여서 다시 list로 만듭니다.
image_list = [os.path.join(origin, x) for x in os.listdir(origin)]

# 만든 image_list에서 첫 번째 파일의 전체 경로를 확인해 봅니다.
image_list[0]    #  =  './data/safety-data/human-accident\\bump'   ...  windows에서는 '/' 이거말고, '\\'이걸로 써야겠습니다.

'./data/safety-data/human-accident\\bump'

<br><br>

---

## 🟢 **[CustomData 사용]** - (예시) 이미지리스트를 내가 만든 데이터셋 클래스에 적용  

In [ ]:
# 이미지에 대한 경로, 일괄적으로 적용할 전처리 이 2가지를 인수로 넣어줌.
dataset = CustomData(image_list, transform) 
# 🔥🔥🔥 어떻게 작용하는 것일까?
DATAS.DWWDWQD()

dataloader = DataLoader(
    dataset=dataset,  # 🔥🔥🔥 어떻게 작용하는 것일까?
    batch_size=25,
    shuffle=True,
    drop_last=False,  # 마지막에 남는 데이터도 사용합니다. (False: 버리지 않음)
)

### 🟡 🔥🔥🔥🔥🔥 DataLoader에서 이미지 데이터 배치 꺼내보기  
- DataLoader에서 이미지 데이터가 어떻게 배치 단위로 나오는지 확인해 보겠습니다.  

In [63]:
dataiter = iter(dataloader) # iterate 
print(dataiter)

batch = next(dataiter)
print(batch)

PermissionError: [Errno 13] Permission denied: './data/safety-data/human-accident\\jam'

In [ ]:
# 배치에는 이미지와 라벨이 함께 들어있으므로, 각각을 images와 labels 변수에 저장.
images, labels = batch

In [ ]:
# 한 배치에 들어있는 이미지와 라벨의 개수를 확인. (batch_size와 동일)
len(images), len(labels)

In [ ]:
# Matplotlib로 첫 번째 데이터 배치 출력
# 이미지가 배치 (N, C, H, W) 형태라고 가정 (예: N=25, C=3, H=224, W=224)
def imshow(img, title=None):
    """이미지 시각화를 위한 함수"""
    img = img.permute(1, 2, 0)  # (C, H, W) -> (H, W, C)
    plt.imshow(img)
    if title is not None:
        plt.title(title)
    plt.axis("off")


# 배치의 첫 번째 이미지 출력
plt.figure(figsize=(10, 10))
for i in range(len(images)):
    plt.subplot(5, 5, i + 1)  # 5x5 그리드 생성 (25개의 이미지)
    imshow(images[i])
    plt.title(f"Label: {labels[i].item()}")
plt.tight_layout()
plt.show()

이 코드를 실행하면, DataLoader가 25개의 전처리된 이미지 데이터(텐서 형태)와 25개의 라벨을 하나의 batch로 묶어서 반환해주는 것을 확인할 수 있습니다.  
이제 이 batch 데이터를 딥러닝 모델에 넣어서 학습을 진행하면 됩니다.  

<br><br><br>

---
---
---


# 🟩 W & B (Weight & Biases)  

In [ ]:
# !pip install wandb

import wandb

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.models as models
import torchvision.transforms as transforms

In [ ]:
wandb.init(
    project="poth_resnet2",  # 프로젝트 이름
    #config.yaml
    config={
        "epochs": 5,
        "batch_size": 25,
        "learning_rate": 0.001,
        "optimizer": "Adam",
        "model": "ResNet18"
    }
)

In [ ]:
# ResNet 모델 정의
model = models.resnet18(pretrained=True)
num_classes = 2  # 데이터셋의 클래스 개수에 맞게 설정
model.fc = nn.Linear(model.fc.in_features, num_classes)

# GPU 사용 여부
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 🔥 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 모델 훈련
num_epochs = wandb.config.epochs

for epoch in range(num_epochs):
    model.train()  # 모델을 학습 모드로 설정
    running_loss = 0.0

    for i, (images, labels) in enumerate(dataloader):
        images, labels = images.to(device), labels.to(device)  # 데이터 GPU로 이동

        optimizer.zero_grad()  # 옵티마이저 초기화
        outputs = model(images)  # 순전파
        loss = criterion(outputs, labels)  # 손실 계산
        loss.backward()  # 역전파
        optimizer.step()  # 가중치 업데이트

        running_loss += loss.item()

        # **WandB 배치 로깅**
        wandb.log({"batch_loss": loss.item()})

    # 에포크 종료 시 평균 손실 로그 기록
    avg_loss = running_loss / len(dataloader)
    wandb.log({"epoch_loss": avg_loss, "epoch": epoch + 1})
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

print("Training complete!")

# 모델 저장 (선택 사항)
torch.save(model.state_dict(), "resnet18_poth_dataset.pth")
wandb.save("resnet18_poth_dataset.pth")
print("Model saved!")